In [2]:
from langchain_anthropic import ChatAnthropicfrom langchain_core.runnables import ConfigurableFieldfrom langchain_core.tools import toolfrom langchain_openai import ChatOpenAI# 定义一个乘法函数@tooldef multiply(x: float, y: float) -> float:    """Multiply 'x' times 'y'."""    return x * y# 定义一个指数函数@tooldef exponentiate(x: float, y: float) -> float:    """Raise 'x' to the 'y'."""    return x**y# 定义一个加法函数@tooldef add(x: float, y: float) -> float:    """Add 'x' and 'y'."""    return x + y# 将三个函数放入一个工具列表中tools = [multiply, exponentiate, add]# 创建一个名为gpt35的ChatOpenAI对象，绑定tools工具列表gpt35 = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0).bind_tools(tools)# 创建一个名为claude3的ChatAnthropic对象，绑定tools工具列表claude3 = ChatAnthropic(model="claude-3-sonnet-20240229").bind_tools(tools)# 使用gpt35和claude3创建一个llm_with_tools对象，使用ConfigurableField配置llm，设置默认键为"gpt35"，另一个键为claude3llm_with_tools = gpt35.configurable_alternatives(    ConfigurableField(id="llm"), default_key="gpt35", claude3=claude3)

# LangGraphLangGraph

In [3]:
# 导入必要的模块import operatorfrom typing import Annotated, Sequence, TypedDict# 导入自定义模块from langchain_core.messages import AIMessage, BaseMessage, HumanMessage, ToolMessagefrom langchain_core.runnables import RunnableLambdafrom langgraph.graph import END, StateGraph# 定义代理状态的类型class AgentState(TypedDict):    messages: Annotated[Sequence[BaseMessage], operator.add]# 判断是否继续执行工作流def should_continue(state):    return "continue" if state["messages"][-1].tool_calls else "end"# 调用模型def call_model(state, config):    return {"messages": [llm_with_tools.invoke(state["messages"], config=config)]}# 调用工具def _invoke_tool(tool_call):    tool = {tool.name: tool for tool in tools}[tool_call["name"]]    return ToolMessage(tool.invoke(tool_call["args"]), tool_call_id=tool_call["id"])tool_executor = RunnableLambda(_invoke_tool)# 调用工具def call_tools(state):    last_message = state["messages"][-1]    return {"messages": tool_executor.batch(last_message.tool_calls)}# 创建状态图workflow = StateGraph(AgentState)workflow.add_node("agent", call_model)  # 添加代理节点workflow.add_node("action", call_tools)  # 添加动作节点workflow.set_entry_point("agent")  # 设置入口点为代理节点workflow.add_conditional_edges(    "agent",    should_continue,    {        "continue": "action",        "end": END,    },)workflow.add_edge("action", "agent")  # 添加从动作节点到代理节点的边graph = workflow.compile()  # 编译工作流图

In [4]:
# 调用图形对象的invoke方法，传入一个包含消息的字典作为参数graph.invoke(    {        "messages": [            # 创建一个HumanMessage对象，传入包含两个数学问题的字符串作为参数            HumanMessage(                "what's 3 plus 5 raised to the 2.743. also what's 17.24 - 918.1241"            )        ]    })

{'messages': [HumanMessage(content="what's 3 plus 5 raised to the 2.743. also what's 17.24 - 918.1241"),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_6yMU2WsS4Bqgi1WxFHxtfJRc', 'function': {'arguments': '{"x": 8, "y": 2.743}', 'name': 'exponentiate'}, 'type': 'function'}, {'id': 'call_GAL3dQiKFF9XEV0RrRLPTvVp', 'function': {'arguments': '{"x": 17.24, "y": -918.1241}', 'name': 'add'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 58, 'prompt_tokens': 168, 'total_tokens': 226}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': 'fp_b28b39ffa8', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-528302fc-7acf-4c11-82c4-119ccf40c573-0', tool_calls=[{'name': 'exponentiate', 'args': {'x': 8, 'y': 2.743}, 'id': 'call_6yMU2WsS4Bqgi1WxFHxtfJRc'}, {'name': 'add', 'args': {'x': 17.24, 'y': -918.1241}, 'id': 'call_GAL3dQiKFF9XEV0RrRLPTvVp'}]),
  ToolMessage(content='300.03770462067547', tool_call_id='call_6yMU2WsS4Bqgi1WxFHxt

In [5]:
# 调用graph的invoke方法，传入参数和配置信息graph.invoke(    {        "messages": [            # 发送一个HumanMessage消息，内容为数学问题            HumanMessage(                "what's 3 plus 5 raised to the 2.743. also what's 17.24 - 918.1241"            )        ]    },    # 传入配置信息，指定configurable的llm为"claude3"    config={"configurable": {"llm": "claude3"}},)

{'messages': [HumanMessage(content="what's 3 plus 5 raised to the 2.743. also what's 17.24 - 918.1241"),
  AIMessage(content=[{'text': "Okay, let's break this down into two parts:", 'type': 'text'}, {'id': 'toolu_01DEhqcXkXTtzJAiZ7uMBeDC', 'input': {'x': 3, 'y': 5}, 'name': 'add', 'type': 'tool_use'}], response_metadata={'id': 'msg_01AkLGH8sxMHaH15yewmjwkF', 'model': 'claude-3-sonnet-20240229', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'input_tokens': 450, 'output_tokens': 81}}, id='run-f35bfae8-8ded-4f8a-831b-0940d6ad16b6-0', tool_calls=[{'name': 'add', 'args': {'x': 3, 'y': 5}, 'id': 'toolu_01DEhqcXkXTtzJAiZ7uMBeDC'}]),
  ToolMessage(content='8.0', tool_call_id='toolu_01DEhqcXkXTtzJAiZ7uMBeDC'),
  AIMessage(content=[{'id': 'toolu_013DyMLrvnrto33peAKMGMr1', 'input': {'x': 8.0, 'y': 2.743}, 'name': 'exponentiate', 'type': 'tool_use'}], response_metadata={'id': 'msg_015Fmp8aztwYcce2JDAFfce3', 'model': 'claude-3-sonnet-20240229', 'stop_reason': 'tool_use', 'stop_sequenc